# Project 2: Enhancer Classification Problem

In [2]:
import gzip
from Bio import SeqIO
from Bio.Seq import Seq
from collections import defaultdict
import random
from sklearn.metrics import roc_auc_score
import itertools
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix

### Loading the data

In [3]:
def load_fasta(file_path):
    sequences = []
    with open(file_path, "r") as fasta_file:
        for record in SeqIO.parse(fasta_file, "fasta"):
            sequences.append({"id": record.id, "sequence": str(record.seq).upper()})
    return pd.DataFrame(sequences)

In [4]:
def load_tsv(file_path):
    with gzip.open(file_path, "rt") as f:
        df_ = pd.read_csv(f, sep="\t")
    return df_

In [5]:
genomes = load_fasta("data/GRCh38.p14.genome.fa")
experiments = load_tsv("data/experiments.tsv.gz")

In [6]:
print(genomes.tail())

             id                                           sequence
701  KI270753.1  GAACACCTAAACGCCCTAAATTCTTTCTTTACTCCATTCTGCTCCT...
702  KI270754.1  GCATAGTCAAGTTTGCAAACCACTGCTCTCGAGTTTTAATTGACAT...
703  KI270755.1  TTAAACGGTTGTTTCACTGCGGGGAAAAGAGTATCCCAAGCTCCTC...
704  KI270756.1  ACGATCTCACTGTGTCACCCAGGTTGGAGTGCGGTGCACAATCTGA...
705  KI270757.1  GTTTTTGAACTGCTAAATAAAAAGAAAGGTTCATCTCTGTGAGACG...


In [7]:
print(experiments.head())

       exp_hier vista_id  allele_id backbone  stage curation_status  \
0  03vn00010001      hs1          0      hZR  e11.5        positive   
1  03v300010001      hs2          0      hZR  e11.5        negative   
2  000100010001      hs3          0      hZR  e11.5        negative   
3  000200010001      hs4          0      hZR  e11.5        positive   
4  03ya00010001      hs5          0      hZR  e11.5        negative   

  description  denominator       tissue tissue_positive  ...  \
0   reference         15.0  lb;hb;nt;cn       3;12;12;8  ...   
1   reference          NaN          NaN             NaN  ...   
2   reference          NaN          NaN             NaN  ...   
3   reference         10.0     mb;hb;nt         10;10;6  ...   
4   reference          NaN          NaN             NaN  ...   

                     coord strand          coordinate_hg38  \
0  chr16:86396481-86397120      +  chr16:86396481-86397120   
1  chr16:85586489-85588130      +  chr16:85586489-85588130   
2 

### Counting k-mers

In [8]:
def count_kmers_for_sequences(sequences, k_):
    bases = ['A', 'T', 'C', 'G']
    all_kmers = {"".join(p) for p in itertools.product(bases, repeat=k_)}
    all_kmers = {min(kmer, str(Seq(kmer).reverse_complement())) for kmer in all_kmers}
    
    feature_vectors = []
    for sequence in sequences:
        sequence = sequence.upper()
        kmer_counts = defaultdict(int)
        n = len(sequence)
        for i in range(n - k_ + 1):
            kmer = sequence[i:i + k_]
            reverse = str(Seq(kmer).reverse_complement())
            kmer_counts[min(kmer, reverse)] += 1

        feature_vector = {kmer: kmer_counts[kmer] / n for kmer in all_kmers}
        feature_vectors.append(feature_vector)
    
    return pd.DataFrame(feature_vectors).fillna(0)

In [9]:
test_sequences = ["ATCATC"]

kmer_features = count_kmers_for_sequences(test_sequences, 3)
sorted_kmer_features = kmer_features[sorted(kmer_features.columns, key=lambda x: -kmer_features[x].sum())]
sorted_kmer_features

,ATC,TCA,ATG,ACA,AAT,CAC,ACC,AAG,AAC,ACG,...,CCC,ACT,AAA,CCG,GGA,GCC,CGC,GTA,AGA,CCA
0,0.333333,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing data

In [10]:
def extract_sequences(data,):
    positive_ = data[data['curation_status'] == 'positive']    
    negative_ = data[data['curation_status'] == 'negative']
    
    positive_ = positive_['seq_hg38'].dropna().apply(str).tolist()
    negative_ = negative_['seq_hg38'].dropna().apply(str).tolist()
    
    return positive_, negative_

In [14]:
def generate_random_negative_sequences(genome_df, positive_sequences, num_sequences):
    random_negatives = []
    genome_seq = "".join(genome_df["sequence"].tolist())
    print("aa")
    print(genome_seq)
    positive_lengths = [len(seq) for seq in positive_sequences]

    while len(random_negatives) < num_sequences:
        seq_length = random.choice(positive_lengths) 
        start = random.randint(0, len(genome_seq) - seq_length)
        seq = genome_seq[start:start + seq_length]
        if 'N' not in seq and seq not in positive_sequences and seq not in random_negatives:
            random_negatives.append(seq)
    return random_negatives

In [15]:
positive, negative = extract_sequences(experiments)

In [16]:
random_negative = generate_random_negative_sequences(genomes, positive, len(positive))

In [35]:
print("Positive: ", len(positive))
print("Negative: ", len(negative))
print("Random negative: ", len(random_negative))

Positive:  2267
Negative:  1913
Difference:  354
Random negative:  2267


In [36]:
def split_train_test(positive_, negative_, n=400):
    positive_df = pd.DataFrame({'sequence': positive_, 'label': 1})
    negative_df = pd.DataFrame({'sequence': negative_, 'label': 0})
    
    test_positives = positive_df.iloc[-n:]
    test_negatives = negative_df.iloc[-n:]
    test_set = pd.concat([test_positives, test_negatives]).sample(frac=1)

    train_positives = positive_df.iloc[:-n]
    train_negatives = negative_df.iloc[:-n]
    train_set = pd.concat([train_positives, train_negatives]).sample(frac=1)
    
    return train_set, test_set

### Traing the model

In [37]:
def evaluate_metrics(y_pred_, y_prob_, y_test):
    accuracy_ = accuracy_score(y_test, y_pred_)
    precision_ = precision_score(y_test, y_pred_)
    recall_ = recall_score(y_test, y_pred_)
    f1_ = f1_score(y_test, y_pred_)
    auc_score_ = roc_auc_score(y_test, y_prob_)
    cm_ = confusion_matrix(y_test, y_pred_)
    return accuracy_, precision_, recall_, f1_, auc_score_, cm_

In [38]:
def grid_search_cross_validation(model_name_, X, y, models_, params_):
    grid_search = GridSearchCV(models_[model_name_], params_[model_name_], cv=10, scoring='roc_auc', n_jobs=1)
    grid_search.fit(X, y)
    return grid_search.best_params_

In [39]:
def train_models(train_set, test_set, models_, params_, k_):
    results_ = []
    X_train = count_kmers_for_sequences(train_set['sequence'], k_)
    y_train = train_set['label']
    X_test = count_kmers_for_sequences(test_set['sequence'], k_)
    y_test = test_set['label']
    for model_name, model in models_.items():
        print("Model: ", model_name)
        best_params = grid_search_cross_validation(model_name, X_train, y_train, models_, params_)
        model.set_params(**best_params)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]

        accuracy, precision, recall, f1, auc_score, cm = evaluate_metrics(y_pred, y_prob, y_test)

        results_.append({
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-score": f1,
            "AUC-ROC": auc_score,
            "Confusion matrix": cm.tolist()
        })
        
    return results_

#### Set-up

In [40]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": xgb.XGBClassifier()
}

In [41]:
params = {
    "Logistic Regression": {
        'C': [0.01, 0.1, 1, 10],            
        'penalty': ['l2'],                 
        'solver': ['liblinear', 'saga']    
    },

    "Random Forest": {
        'n_estimators': [100, 200],    
        'max_depth': [None, 20],        
        'min_samples_split': [2, 5]  
    },

    "XGBoost": {
        'learning_rate': [0.1, 0.3],
        'max_depth': [4, 6, 8]                               
    }
}

In [42]:
k_values = [3, 4, 5]

#### Negatives from experiments.tsv

In [43]:
train, test = split_train_test(positive, negative)
results_exp = {}

In [44]:
for k in k_values:
    print(f"\nTraining models for k={k}\n")
    results_k = train_models(train, test, models, params, k_=k)
    results_exp[k] = results_k  


Training models for k=3
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost

Training models for k=4
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost

Training models for k=5
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost


In [45]:
for k, results_k in results_exp.items():
    print(f"Results for k={k}")
    df = pd.DataFrame(results_k)
    print(df)

Results for k=3
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.50875   0.504648  0.9500  0.659150  0.564916   
1        Random Forest   0.52750   0.519504  0.7325  0.607884  0.558847   
2              XGBoost   0.53125   0.523191  0.7050  0.600639  0.541753   

           Confusion matrix  
0    [[27, 373], [20, 380]]  
1  [[129, 271], [107, 293]]  
2  [[143, 257], [118, 282]]  
Results for k=4
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.50125   0.500649   0.965  0.659266  0.564747   
1        Random Forest   0.53250   0.521036   0.805  0.632613  0.572219   
2              XGBoost   0.52125   0.515654   0.700  0.593849  0.562359   

           Confusion matrix  
0    [[15, 385], [14, 386]]  
1   [[104, 296], [78, 322]]  
2  [[137, 263], [120, 280]]  
Results for k=5
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.50500   

#### Random sequences from the entire genome

In [46]:
train, test = split_train_test(positive, random_negative)
results_rnd = {}

In [47]:
for k in k_values:
    print(f"\nTraining models for k={k}\n")
    results_k = train_models(train, test, models, params, k_=k)
    results_rnd[k] = results_k 


Training models for k=3
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost

Training models for k=4
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost

Training models for k=5
Model:  Logistic Regression
Model:  Random Forest
Model:  XGBoost


In [48]:
for k, results_k in results_rnd.items():
    print(f"Results for k={k}")
    df = pd.DataFrame(results_k)
    print(df)

Results for k=3
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.68375   0.719403  0.6025  0.655782  0.761706   
1        Random Forest   0.70000   0.729885  0.6350  0.679144  0.766550   
2              XGBoost   0.68875   0.718841  0.6200  0.665772  0.770531   

          Confusion matrix  
0  [[306, 94], [159, 241]]  
1  [[306, 94], [146, 254]]  
2  [[303, 97], [152, 248]]  
Results for k=4
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.67750   0.716463  0.5875  0.645604  0.770281   
1        Random Forest   0.71750   0.755882  0.6425  0.694595  0.788372   
2              XGBoost   0.72625   0.756374  0.6675  0.709163  0.804838   

          Confusion matrix  
0  [[307, 93], [165, 235]]  
1  [[317, 83], [143, 257]]  
2  [[314, 86], [133, 267]]  
Results for k=5
                 Model  Accuracy  Precision  Recall  F1-score   AUC-ROC  \
0  Logistic Regression   0.67250   0.709091